In [1]:
import pandas as pd
import json

dashboard = pd.ExcelFile("data.xlsx")

In [2]:
data = pd.read_excel(dashboard,"Statistics")

aes = pd.read_excel(dashboard,"AEs")

referrals = pd.read_excel(dashboard, "Referrals")

partnerDistricts = pd.read_excel("partnerDistrict.xlsx")

In [3]:
data['totalMCs'] = data['mc15-19'] + data['mc20-24'] +data['mc25-29']+data['mc30-34']+data['mc35-39'] + data['mc40-44'] + data['mc45-49']+ data['mc50']

data

,AERecordingMonth,AERecordingSite,AERecordingYear,District,ReferrelRecordingMonth,ReferrelRecordingSite,ReferrelRecordingYear,Site_Name,TotalMCsBYMethod,aeComments,...,total_surgicalReusable.5,total_surgicalReusable.6,total_surgicalReusable.7,total_surgicalReusable.8,totalhivNegative,totalhivNegativeNC,uncircumcisedClientsForHTS,vmmc_number,year,facilityType
0,NaN,NaN,NaN,GURUVE,December,Gota,2024.0,GURUVE - 100553 - DISTRICT HOSPITAL,15,NaN,...,NaN,OK,OK,NaN,15,1,1,NaN,2024,NaN
1,December,Svita Clinic,2024.0,MBERENGWA,December,Svita Clinic,2024.0,MUSUME - 101157 - MISSION HOSPITAL,43,Client is healing,...,NaN,NaN,OK,NaN,43,0,0,585858.0,2024,NaN
2,NaN,NaN,NaN,MBERENGWA,December,Neta Clinic,2024.0,MUSUME - 101157 - MISSION HOSPITAL,45,NaN,...,NaN,NaN,OK,NaN,44,0,0,NaN,2024,NaN
3,December,Musume Hospital,2024.0,MBERENGWA,NaN,NaN,NaN,MUSUME - 101157 - MISSION HOSPITAL,25,Client is healing,...,NaN,OK,OK,NaN,25,0,1,9857.0,2024,NaN
4,December,Chipuriro,2024.0,GURUVE,NaN,NaN,NaN,GURUVE - 100553 - DISTRICT HOSPITAL,46,Client is healing,...,OK,NaN,OK,OK,44,0,0,8886.0,2024,NaN
5,NaN,NaN,NaN,GOKWE SOUTH,NaN,NaN,NaN,GOKWE - 100513 - DISTRICT HOSPITAL,24,NaN,...,NaN,OK,OK,NaN,24,0,1,NaN,2024,Outreach
6,December,Guruve District Hospital,2024.0,GURUVE,December,Guruve District Hospital,2024.0,GURUVE - 100553 - DISTRICT HOSPITAL,34,Client is healing,...,OK,NaN,OK,OK,32,0,0,456.0,2024,NaN
7,NaN,NaN,NaN,MATOBO,NaN,NaN,NaN,MAPHISA - 100890 - DISTRICT HOSPITAL,43,NaN,...,OK,OK,OK,OK,43,0,1,NaN,2024,Outreach
8,NaN,NaN,NaN,GOKWE SOUTH,December,Gokwe South District Hospital,2024.0,GOKWE - 100513 - DISTRICT HOSPITAL,73,NaN,...,OK,NaN,OK,OK,69,0,0,NaN,2024,Static
9,December,Gwanyika Rural Health Centre,2024.0,GOKWE SOUTH,December,Gwanyika Rural Health Centre,2024.0,GOKWE - 100513 - DISTRICT HOSPITAL,37,Client is healing,...,OK,NaN,OK,OK,37,0,0,345534.0,2024,Outreach


In [4]:
aes

,AERecordingMonth,AERecordingSite,AERecordingYear,District,cancelthisAEReport,aeComments,ae_classification,ae_type_code,circumcising_cadre,client_age,date_ae_identified,mcMethod,vmmc_number
0,November,Krima Clinic,2024.0,GOKWE SOUTH,NaN,Client healing,Severe,COA,Nurse,25,2024-11-10,Surgical,467233
1,November,Krima Clinic,2024.0,GOKWE SOUTH,NaN,Client healing,Moderate,BL,Nurse,18,2024-11-08,Surgical,245555
2,November,Mbembeswana,2024.0,MATOBO,NaN,Client is healing,Severe,COA,NaN,25,2024-10-03,Surgical,233446
3,November,Mbembeswana,2024.0,MATOBO,NaN,Client is healing,Moderate,SSE,Nurse,35,2024-10-03,Surgical,3467
4,NaN,NaN,NaN,MATOBO,NaN,Client Healimg at home,Severe,COA,Nurse,25,2024-09-03,Surgical,234455
5,NaN,NaN,NaN,MATOBO,NaN,Client Healing,Moderate,SSE,Nurse,23,2024-12-03,Surgical,578988
6,December,Svita Clinic,2024.0,MBERENGWA,NaN,Client is healing,Severe,COA,Nurse,16,2024-12-07,Surgical,585858
7,December,Musume Hospital,2024.0,MBERENGWA,NaN,Client is healing,Moderate,COA,Nurse,25,2024-12-18,Surgical,9857
8,December,Chipuriro,2024.0,GURUVE,NaN,Client is healing,Severe,COA,Nurse,25,2024-12-06,Surgical,8886
9,December,Guruve District Hospital,2024.0,GURUVE,NaN,Client is healing,Moderate,COA,Doctor,25,2024-12-02,Surgical,456


In [5]:
# Ensure Month-Year columns are strings for easy comparison
data['year'] = data['year'].astype(str)
aes['AERecordingYear'] = aes['AERecordingYear'].astype(str)

data['recordingMonth'] = data['recordingMonth'].astype(str)
aes['AERecordingMonth'] = aes['AERecordingMonth'].astype(str)

# Transform df1 to include the list object
json_data = []

for _, row in data.iterrows():
    row_data = row.to_dict()
    print(type(row['totalAES']))
    if row['totalAES'] >= 1:
        # Filter df2 to find matching rows
        recordedAEs = aes[
            (aes['AERecordingSite'] == row['facilityName']) &
            (aes['AERecordingMonth'] == row['recordingMonth']) &
            (aes['AERecordingYear'] == row['year'])
        ]

        print(recordedAEs)
        
        # Add list object to the row if matches are found
        row_data['matchingAES'] = recordedAEs.to_dict(orient='records')
    else:
        # Add an empty list if no matches
        row_data['matchingAES'] = []


    if row['referralToOtherServices'] >= 1:
        # Filter df2 to find matching rows
        otherReferrals = referrals[
            (referrals['ReferrelRecordingSite'] == row['facilityName']) &
            (referrals['ReferrelRecordingMonth'] == row['recordingMonth']) &
            (referrals['ReferrelRecordingYear'] == row['year'])
        ]


        # Add list object to the row if matches are found
        row_data['otherReferrals'] = otherReferrals.to_dict(orient='records')
    else:
        # Add an empty list if no matches
        row_data['otherReferrals'] = []
    
    # Append to the final JSON structure
    json_data.append(row_data)

# Convert to JSON
result_json = json.dumps(json_data, indent=4)

output_file = "data.json"

# Save the JSON data to the file
with open(output_file, "w") as file:
    json.dump(json_data, file, indent=4)

# Print the JSON
print(result_json)

<class 'int'>
<class 'int'>
Empty DataFrame
Columns: [AERecordingMonth, AERecordingSite, AERecordingYear, District, cancelthisAEReport, aeComments, ae_classification, ae_type_code, circumcising_cadre, client_age, date_ae_identified, mcMethod, vmmc_number]
Index: []
<class 'int'>
<class 'int'>
Empty DataFrame
Columns: [AERecordingMonth, AERecordingSite, AERecordingYear, District, cancelthisAEReport, aeComments, ae_classification, ae_type_code, circumcising_cadre, client_age, date_ae_identified, mcMethod, vmmc_number]
Index: []
<class 'int'>
Empty DataFrame
Columns: [AERecordingMonth, AERecordingSite, AERecordingYear, District, cancelthisAEReport, aeComments, ae_classification, ae_type_code, circumcising_cadre, client_age, date_ae_identified, mcMethod, vmmc_number]
Index: []
<class 'int'>
<class 'int'>
Empty DataFrame
Columns: [AERecordingMonth, AERecordingSite, AERecordingYear, District, cancelthisAEReport, aeComments, ae_classification, ae_type_code, circumcising_cadre, client_age, dat